In [1]:
# Imports
load('prismatic_envelope.sage')
load('homology.sage')
load('precision.py')

### User-defined input

In [2]:
# The prime p
p=2

# The motivic weight i in F_p(i)^red
i=2

# The power of the uniformizer n
n=3

# The residual degree f
# The present code only supports totally ramified extensions of Qp,
# i.e., where f=1
f=1

Fprec,total_precision=target_total_precision(p,i,n)

print("total p-adic precision is {}".format(total_precision))
print("Fprec is {}".format(Fprec))

# The coefficient ring W
if f==1:
    W=Zp(p,total_precision,type='capped-abs',print_mode='digits',show_prec=False)
else:
    W=Zq(p**f,total_precision,names='a',type='capped-abs',print_mode='series',show_prec=False)
    
# The Breuil-Kisin ring A
A.<z>=PowerSeriesRing(W,Fprec)

# The Eisenstein polynomial E
# USER DEFINED
E=A(z+p)

total p-adic precision is 20
Fprec is 6


### The main calculation

In [3]:
#%%capture
# Suppresses some Python warnings and SAGE variable injections

# The normalized Eisenstein polynomial
# The normalization is to bring the Eisenstein polynomial into the form E(0)=p
E=eisenstein_normalization(p,E)

if i-p+1>0:
    a_syn0,a_syn1,a_nablaN,a_nablaP,a_gens_N0,a_gens_P0,a_gens_N1,a_gens_P1=syntomic_matrices(p,i-p+1,n,E,total_precision,Fprec,debug=False)
    b_syn0,b_syn1,b_nablaN,b_nablaP,b_gens_N0,b_gens_P0,b_gens_N1,b_gens_P1=syntomic_matrices(p,i,n,E,total_precision,Fprec,debug=False)

<string>:679: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:698: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:715: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:725: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:739: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:758: DeprecationWarning: __getitem__

In [4]:
class SyntomicComplex:
    def __init__(self,p,i,n,E,total_precision,Fprec,debug=False):
        self.syn0,self.syn1,self.nablaN,self.nablaP,self.gens_0,gens_P0,gens_N1,self.gens_2=syntomic_matrices(p,i,n,E,total_precision,Fprec,debug=False)
        self.gens_1=gens_P0+gens_N1
        self.gens={}
        self.gens[0]=self.gens_0
        self.gens[1]=self.gens_1
        self.gens[2]=self.gens_2
        d0=block_matrix([[self.syn0],[self.nablaN]])
        d1=block_matrix([[self.nablaP,-self.syn1]])
        self.complex=ChainComplex({0:d0,1:d1})
    
    def compute_mod_p_homology(self):
        self.complex_mod_p=ChainComplex({0:Matrix(GF(p),self.complex.differential()[0]),1:Matrix(GF(p),self.complex.differential()[1])})
        self.homology_mod_p=Homology(self.complex_mod_p)
        self.homology_mod_p_orders=self.homology_mod_p.homology_orders
        self.homology_mod_p_dimensions={k:len(v) for k,v in self.homology_mod_p_orders.items()}
        self.homology_mod_p_generators={}
        for k,v in self.homology_mod_p.homology_representatives.items():            
            self.homology_mod_p_generators[k]=[]
            for t in range(v.ncols()):
                gen_name=''
                for s in range(v.nrows()):
                    if v[s,t] != 0:
                        gen_name+='+'+str(v[s,t])+'*'+self.gens[k][s]
                self.homology_mod_p_generators[k].append(gen_name)
            print("The dimension of the mod p cohomology in degree {} is {}.".format(
                k,self.homology_mod_p_dimensions[k]))
            print("The generators of the mnod p cohomology are ")
            print(self.homology_mod_p_generators[k])
            print('\n')

In [5]:
syn_cplx=SyntomicComplex(p,i-p+1,n,E,total_precision,Fprec,debug=False)

<string>:679: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:698: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:715: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:725: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:739: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:758: DeprecationWarning: __getitem__

In [6]:
syn_cplx.compute_mod_p_homology()

The dimension of the mod p cohomology in degree 0 is 2.
The generators of the mnod p cohomology are 
['+1*E^1z+1*E^0f0+1*E^0zf0', '+1*E^1z']


The dimension of the mod p cohomology in degree 1 is 2.
The generators of the mnod p cohomology are 
['+1*z∂', '+1*z∂+1*f0∂+1*E^0∇z+1*E^0z∇z']


The dimension of the mod p cohomology in degree 2 is 0.
The generators of the mnod p cohomology are 
[]




In [7]:
ascii_art(syn_cplx.complex_mod_p)

                                        [0 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [0 1 1 0 0]      
                                        [0 1 0 1 0]      
                                        [0 0 0 0 1]      
            [1 0 0 0 0 1 0 0 0 0]       [0 0 0 0 0]      
            [0 0 0 0 0 1 1 0 0 0]       [0 0 0 0 0]      
            [1 0 1 0 0 0 0 1 0 0]       [0 1 1 0 0]      
            [0 0 0 0 0 1 1 0 1 0]       [0 0 0 0 0]      
            [0 0 0 0 1 0 0 0 0 1]       [0 0 0 0 1]      
 0 <-- C_2 <---------------------- C_1 <------------ C_0 <-- 0 

In [8]:
a_d0=block_matrix([[a_syn0],[a_nablaN]])
a_d1=block_matrix([[a_nablaP,-a_syn1]])
a_d0_Fp=Matrix(GF(p),a_d0)
a_d1_Fp=Matrix(GF(p),a_d1)
a_C=ChainComplex({0:a_d0_Fp,1:a_d1_Fp})
a_C.homology()

{0: Vector space of dimension 2 over Finite Field of size 2,
 1: Vector space of dimension 2 over Finite Field of size 2,
 2: Vector space of dimension 0 over Finite Field of size 2}

In [9]:
b_d0=block_matrix([[b_syn0],[b_nablaN]])
b_d1=block_matrix([[b_nablaP,-b_syn1]])
b_d0_Fp=Matrix(GF(p),b_d0)
b_d1_Fp=Matrix(GF(p),b_d1)
b_C=ChainComplex({0:b_d0_Fp,1:b_d1_Fp})
b_C.homology()

{0: Vector space of dimension 2 over Finite Field of size 2,
 1: Vector space of dimension 3 over Finite Field of size 2,
 2: Vector space of dimension 1 over Finite Field of size 2}

In [10]:
ascii_art(a_C)

                                        [0 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [0 1 1 0 0]      
                                        [0 1 0 1 0]      
                                        [0 0 0 0 1]      
            [1 0 0 0 0 1 0 0 0 0]       [0 0 0 0 0]      
            [0 0 0 0 0 1 1 0 0 0]       [0 0 0 0 0]      
            [1 0 1 0 0 0 0 1 0 0]       [0 1 1 0 0]      
            [0 0 0 0 0 1 1 0 1 0]       [0 0 0 0 0]      
            [0 0 0 0 1 0 0 0 0 1]       [0 0 0 0 1]      
 0 <-- C_2 <---------------------- C_1 <------------ C_0 <-- 0 

In [11]:
ascii_art(b_C)

                                        [0 0 0 0 0]      
                                        [1 0 0 0 0]      
                                        [1 0 0 0 0]      
                                        [0 0 1 0 0]      
                                        [0 0 0 1 0]      
            [1 0 0 0 0 0 0 0 0 0]       [0 0 0 0 0]      
            [0 0 0 0 0 0 0 0 0 0]       [1 0 0 0 0]      
            [0 0 1 0 0 0 1 0 0 0]       [0 0 1 0 0]      
            [0 1 0 0 0 0 1 1 1 0]       [0 0 1 0 0]      
            [0 0 1 0 1 0 0 0 0 1]       [1 0 0 1 0]      
 0 <-- C_2 <---------------------- C_1 <------------ C_0 <-- 0 

# Computing v1

In [12]:
v1N0,v1P0,v1N1,v1P1=v1_matrices(p,i,n,E,total_precision,Fprec,debug=False)

<string>:832: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:841: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:854: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:868: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.












z*d_tilde^3
(10*z + z^2 + O(z^6))*d_tilde^2
z^2*d_tilde^3
(10*z^2 + O(z^6))*d_tilde^2 + ((10 + z + O(z^3))*f0)*d_tilde
f0*d_tilde^2
((10 + z + O(z^3))*f0)*d_tilde
z*f0*d_tilde^2
((10*z + z^2 + O(z^3))*f0)*d_tilde
z^2*f0*d_tilde^2
((10*z^2 + O(z^3))*f0)*d_tilde


<string>:880: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:889: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:901: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:913: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.


In [13]:
v1N0_Fp=Matrix(GF(p),v1N0)
v1P0_Fp=Matrix(GF(p),v1P0)
v1N1_Fp=Matrix(GF(p),v1N1)
v1P1_Fp=Matrix(GF(p),v1P1)

In [14]:
v1N0_Fp

[0 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 0]
[0 1 1 0 0]
[0 0 0 1 0]

In [15]:
v1P0_Fp

[0 0 0 0 0]
[0 0 0 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]

In [16]:
v1N1_Fp

[0 0 0 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 0 0]
[0 0 1 0 0]

In [17]:
v1P1_Fp

[0 0 0 0 0]
[0 0 0 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]

In [18]:
v1_0=Matrix(GF(p),v1N0)
v1_1=Matrix(GF(p),block_matrix([[v1P0,0],[0,v1N1]]))
v1_2=Matrix(GF(p),v1P1)

In [19]:
v1_0

[0 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 0]
[0 1 1 0 0]
[0 0 0 1 0]

In [20]:
v1_1

[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0]

In [21]:
v1_2

[0 0 0 0 0]
[0 0 0 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]

In [22]:
v1_1*a_C.differential()[0]

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 1 1 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 1 1 0 0]

In [23]:
b_C.differential()[0]*v1_0

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 1 1 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 1 1 0 0]

In [24]:
v1_2*a_C.differential()[1]==b_C.differential()[1]*v1_1

True

In [25]:
v1={0:v1_0,1:v1_1,2:v1_2}
ha_C = Homology(a_C)
hb_C = Homology(b_C)
mor = MorphismHomology(ha_C, hb_C, v1)

for h in mor.hF.values():
    print(ascii_art(h))
    print('\n')

[0 1]
[1 0]


[1 1]
[1 0]
[0 0]


[]




In [26]:
ascii_art(ha_C.complex_smith)

                                        [0 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [---------]      
            [1 0 0 0 0 0 0 0 0 0]       [1 0 0 0 0]      
            [0 1 0 0 0 0 0 0 0 0]       [0 1 0 0 0]      
            [0 0 1 0 0 0 0 0 0 0]       [0 0 1 0 0]      
            [0 0 0 1 0 0 0 0 0 0]       [0 0 0 0 0]      
            [0 0 0 0 1 0 0 0 0 0]       [0 0 0 0 0]      
 0 <-- C_2 <---------------------- C_1 <------------ C_0 <-- 0 

In [27]:
ascii_art(hb_C.complex_smith)

                                        [0 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [0 0 0 0 0]      
                                        [---------]      
                                        [1 0 0 0 0]      
            [1 0 0 0 0 0 0 0 0 0]       [0 1 0 0 0]      
            [0 1 0 0 0 0 0 0 0 0]       [0 0 1 0 0]      
            [0 0 1 0 0 0 0 0 0 0]       [0 0 0 0 0]      
            [0 0 0 1 0 0 0 0 0 0]       [0 0 0 0 0]      
            [0 0 0 0 0 0 0 0 0 0]       [0 0 0 0 0]      
 0 <-- C_2 <---------------------- C_1 <------------ C_0 <-- 0 

In [28]:
ha_C.homology_representatives[1]

[0 1]
[1 0]
[0 1]
[0 0]
[0 0]
[0 1]
[0 1]
[0 0]
[0 0]
[0 0]

In [29]:
ha_C.homology_orders[1]

[0, 0]

In [30]:
red_0=block_matrix([[-a_C.differential()[0]],[-v1[0]]])
red_1=block_matrix([[-a_C.differential()[1],0],[-v1[1],b_C.differential()[0]]])
red_2=block_matrix([[-v1[2],b_C.differential()[1]]])
reduced_complex=ChainComplex({-1:red_0,0:red_1,1:red_2})
ascii_art(reduced_complex)

                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
            [0 0 0 0 0|1 0 0 0 0 0 0 0 0 0]      
            [0 0 0 0 0|0 0 0 0 0 0 0 0 0 0]      
            [1 0 0 0 0|0 0 1 0 0 0 1 0 0 0]      
            [0 1 0 0 0|0 1 0 0 0 0 1 1 1 0]      
            [0 0 1 0 0|0 0 1 0 1 0 0 0 0 1]      
 0 <-- C_2 <-------------------------------- C_1 

 [1 0 0 0 0 1 0 0 0 0|0 0 0 0 0]       [0 0 0 0 0]       
 [0 0 0 0 0 1 1 0 0 0|0 0 0 0 0]       [0 0 0 0 0]       
 [1 0 1 0 0 0 0 1 0 0|0 0 0 0 0] 

In [31]:
reduced_homology=Homology(reduced_complex)

In [32]:
reduced_homology.homology_orders

{-1: [], 0: [], 1: [0], 2: [0]}

In [33]:
M=Matrix(ZZ,[[1,2,3],[4,5,6],[7,8,9]]);M

[1 2 3]
[4 5 6]
[7 8 9]

In [34]:
len(M.columns()[0])

3

In [35]:
M.columns()[0][2]

7